In [5]:
from Paper_Results import TVSSegmentation #as pr
import Segmantation_lib as sg
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
trSegem=TVSSegmentation(tvs="g",season=1,episode=1)

You have chosen :
Tv-Series: Game of Thrones
Season:  1
Episode: 1
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season01.Episode01.en.srt


In [7]:
framesFeatures, timeStamp,shot=trSegem.load_Data()

In [8]:
manScene_boundry=trSegem.load_ManAnnotation2()
_,manShot_boundry=sg.getShots(shot)
manClstr_ref,sceneMakar=trSegem.frameGroundClusters(manShot_boundry)
manScene_boundry=[i/1000 for i in manScene_boundry]
avgFeatures=trSegem.getAvgFeatures(framesFeatures,timeStamp,manShot_boundry)
randSampleFeatures,uniSmplFeatures=trSegem.getRandomFeat(framesFeatures,timeStamp,manShot_boundry)

In [9]:
type(manScene_boundry)

list

In [14]:
clstAlgo=['specteral','specteral','specteral']
#kc=[[7,5,6,4],[6,4,3,3],[9,7,7,5]]
#kc=[[7,5,4,3],[7,5,4,3],[7,5,4,3]]#Game of thrones
k_Late=[8,9,7,6,8,6,10,4]
c_late=[4,4,4,5,5,5]
k_Ealry=[11,6,5,5,3,4,11]
c_Early=[3,4,4,4,2,3,2]
w_Early=[124,363,281,320,598,280,234]
w_late=[398,584,300,378,584,581,515]
dfFeat=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df1=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
dfBest=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision','season','episode'))

# Late Fusion

In [15]:
avgFeatures=np.array(avgFeatures)
randSampleFeatures=np.array(randSampleFeatures)
randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
uniSmplFeatures=np.array(uniSmplFeatures)
uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
shot_TxtEmb=np.array(shot_TxtEmb)
shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

dist_AvgFeat=trSegem.getSimilarityMatrix(avgFeatures,'cosine')
dist_RandFeat=trSegem.getSimilarityMatrix(randSampleFeatures,'cosine')
dist_UniFeat=trSegem.getSimilarityMatrix(uniSmplFeatures,'cosine')
dist_shot_TxtEmb=trSegem.getSimilarityMatrix(shot_TxtEmb,'cosine')

manShot_boundry1=[i/1000 for i in manShot_boundry]
shottimenp=np.array(manShot_boundry1[0:len(dist_AvgFeat)])
shottimenp=shottimenp/np.linalg.norm(shottimenp)
shottimenp=shottimenp.reshape(-1,1)
dist_shottimenp=trSegem.getSimilarityMatrix(shottimenp,'cosine')
dist_Visual=dist_AvgFeat
dist_AvgFeat=dist_AvgFeat+dist_shot_TxtEmb/2
dist_RandFeat=dist_RandFeat+dist_shot_TxtEmb/2
dist_UniFeat=dist_UniFeat+dist_shot_TxtEmb/2

dist_AvgFeat_Time=dist_AvgFeat+dist_shottimenp/2
dist_RandFeat_Time=dist_RandFeat+dist_shottimenp/2
dist_UniFeat_Time=dist_UniFeat+dist_shottimenp/2

# Early Fusion

In [17]:
avgFeatures=np.array(avgFeatures)
randSampleFeatures=np.array(randSampleFeatures)
randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
uniSmplFeatures=np.array(uniSmplFeatures)
uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
shot_TxtEmb=np.array(shot_TxtEmb)
shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

conctAvgFeat=np.concatenate((avgFeatures,shot_TxtEmb),axis=1)
conctRandFeat=np.concatenate((randSampleFeatures,shot_TxtEmb),axis=1)
conctUniFeat=np.concatenate((uniSmplFeatures,shot_TxtEmb),axis=1)
conctAvgFeat.shape, conctRandFeat.shape

shottimenp=np.array(manShot_boundry[0:len(conctAvgFeat)])
conctAvgFeat_Time=np.column_stack((conctAvgFeat,shottimenp))
conctRandFeat_Time=np.column_stack((conctRandFeat,shottimenp))
conctUniFeat_Time=np.column_stack((conctUniFeat,shottimenp))
conctAvgFeat_Time.shape, conctRandFeat_Time.shape,shottimenp.shape

dist_AvgFeat=trSegem.getSimilarityMatrix(conctAvgFeat,'cosine')
dist_RandFeat=trSegem.getSimilarityMatrix(conctRandFeat,'cosine')
dist_UniFeat=trSegem.getSimilarityMatrix(conctUniFeat,'cosine')
dist_AvgFeat_Time=trSegem.getSimilarityMatrix(conctAvgFeat_Time,'cosine')
dist_RandFeat_Time=trSegem.getSimilarityMatrix(conctRandFeat_Time,'cosine') 
dist_UniFeat_Time=trSegem.getSimilarityMatrix(conctUniFeat_Time,'cosine')

In [18]:
#for i, clAl in enumerate(clstAlgo):
fusk=k_Ealry
fusC=c_Early
fusW=w_Early
clAl='sp'
visClusters=trSegem.getClusters(40,clAl,dist_Visual)
avgFeatClusters=trSegem.getClusters(40,clAl,dist_AvgFeat)
randFeatClusters=trSegem.getClusters(40,clAl,dist_RandFeat)
uniFeatClusters=trSegem.getClusters(40,clAl,dist_UniFeat)
avgFeatClusters=avgFeatClusters.tolist()
randFeatClusters=randFeatClusters.tolist()
uniFeatClusters=uniFeatClusters.tolist()
avgFeatClusters_Time=trSegem.getClusters(40,clAl,dist_AvgFeat_Time)
randFeatClusters_Time=trSegem.getClusters(40,clAl,dist_RandFeat_Time)
uniFeatClusters_Time=trSegem.getClusters(40,clAl,dist_UniFeat_Time)
avgFeatClusters_Time=avgFeatClusters_Time.tolist()
randFeatClusters_Time=randFeatClusters_Time.tolist()
uniFeatClusters_Time=uniFeatClusters_Time.tolist()
visClusters=visClusters.tolist()

sepVisual=sg.sequenceSegmentation(visClusters,fusk[0],fusC[0])
sepPos=sg.sequenceSegmentation(avgFeatClusters,fusk[1],fusC[1])
sepPos1=sg.sequenceSegmentation(randFeatClusters,fusk[2],fusC[2])
sepPos2=sg.sequenceSegmentation(uniFeatClusters,fusk[3],fusC[3])
sceneTime=[]
sceneTime1=[]
sceneTime2=[]
sepPos_Time=sg.sequenceSegmentation(avgFeatClusters_Time,fusk[4],c_Early[4])
sepPos1_Time=sg.sequenceSegmentation(randFeatClusters_Time,fusk[5],c_Early[5])
sepPos2_Time=sg.sequenceSegmentation(uniFeatClusters_Time,fusk[6],c_Early[6])

"""sepVisual,_=sg.seqSegUsingSet(visClusters,fusW[1])
sepPos,_=sg.seqSegUsingSet(avgFeatClusters,fusW[1])
sepPos1,_=sg.seqSegUsingSet(randFeatClusters,fusW[2])
sepPos2,_=sg.seqSegUsingSet(uniFeatClusters,fusW[3])
sepPos_Time,_=sg.seqSegUsingSet(avgFeatClusters_Time,fusW[4])
sepPos1_Time,_=sg.seqSegUsingSet(randFeatClusters_Time,fusW[5])
sepPos2_Time,_=sg.seqSegUsingSet(uniFeatClusters_Time,fusW[6])
"""
sceneTime_Time=[]
sceneTime1_Time=[]
sceneTime2_Time=[]
sceneTimeVis=[]
for i in sepVisual:
    sceneTimeVis.append(manShot_boundry[i])
for i in sepPos:
    sceneTime.append(manShot_boundry[i])
for i in sepPos1:
    sceneTime1.append(manShot_boundry[i])
for i in sepPos2:
    sceneTime2.append(manShot_boundry[i])
for i in sepPos2_Time:
    sceneTime2_Time.append(manShot_boundry[i])
for i in sepPos_Time:
    sceneTime_Time.append(manShot_boundry[i])
for i in sepPos1_Time:
    sceneTime1_Time.append(manShot_boundry[i])
try:
    print('Auto',clAl,':',len(sepVisual),len(sepPos),len(sepPos1),len(sepPos2),len(sepPos_Time),len(sepPos1_Time),len(sepPos2_Time),'the Manual:',len(manScene_boundry))
    tMan,tMan_bin,tMan_bin_old,sceneTime_man_bin=trSegem.getTruthValue_Man(manShot_boundry,manScene_boundry)
    len(tMan),len(tMan_bin),len(tMan_bin_old),len(sceneTime_man_bin)
    tAutoVis,tAuto_binVis,tAuto_bin_old_Vis,_=trSegem.getTruthValues_Auto(sepVisual,manShot_boundry)
    tAuto,tAuto_bin,tAuto_bin_old,sceneTime_Auto_bin=trSegem.getTruthValues_Auto(sepPos,manShot_boundry)
    tAuto1,tAuto_bin1,tAuto_bin_old1,sceneTime_Auto_bin1=trSegem.getTruthValues_Auto(sepPos1,manShot_boundry)
    tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,sceneTime_Auto_bin_Time=trSegem.getTruthValues_Auto(sepPos_Time,manShot_boundry)
    tAuto2,tAuto_bin2,tAuto_bin_old2,sceneTime_Auto_bin2=trSegem.getTruthValues_Auto(sepPos2,manShot_boundry)
    tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2,sceneTime_Auto_bin_Time2=trSegem.getTruthValues_Auto(sepPos2_Time,manShot_boundry)
    tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,sceneTime_Auto_bin1_Time=trSegem.getTruthValues_Auto(sepPos1_Time,manShot_boundry)

    truth_valueList=[tAuto,tAuto_bin,tAuto_bin_old,tAuto1,tAuto_bin1,tAuto_bin_old1,tAuto2,tAuto_bin2,tAuto_bin_old2]
    truth_valueList_Time=[tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2]
    truth_valueListMan=[tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old]

    results=trSegem.saveEvaluations(tMan,tAutoVis,rType='macro')
    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfFeat = dfFeat.append(row)
    results=trSegem.saveEvaluations(tMan_bin,tAuto_binVis,rType='binary')
    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfFeat = dfFeat.append(row)

    for i in range(len(truth_valueList)):
        if i%3==0:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='macro')
        else:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='binary')
        row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
        df = df.append(row)
        df_tmp=df_tmp.append(row)

    row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    df_tmp=df_tmp.drop([clAl])

    for i in range(len(truth_valueList_Time)):
        if i%3==0:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='macro')
        else:
            results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='binary')
        row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
        df1 = df1.append(row)
        df_tmp=df_tmp.append(row)

    row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
    dfBest=dfBest.append(row1)
    df_tmp=df_tmp.drop([clAl])
except:
    print('there was an error')

Auto sp : 39 33 34 32 27 30 32 the Manual: 27


In [19]:
print(avgFeatClusters)

[9, 17, 17, 17, 14, 3, 17, 17, 9, 35, 20, 6, 6, 35, 6, 35, 6, 35, 35, 35, 6, 35, 35, 35, 35, 35, 35, 35, 20, 35, 35, 35, 35, 35, 35, 6, 12, 35, 14, 10, 14, 35, 35, 35, 35, 38, 38, 35, 38, 35, 38, 35, 35, 35, 10, 35, 35, 35, 35, 35, 35, 35, 6, 6, 35, 6, 35, 6, 38, 35, 38, 35, 38, 6, 6, 37, 35, 37, 35, 35, 6, 37, 35, 37, 35, 35, 35, 35, 6, 35, 35, 35, 35, 23, 35, 35, 35, 35, 35, 6, 37, 35, 37, 6, 37, 23, 6, 23, 23, 37, 23, 23, 37, 35, 37, 34, 34, 9, 9, 9, 9, 9, 9, 9, 9, 17, 9, 9, 9, 9, 9, 17, 9, 9, 9, 9, 9, 9, 9, 34, 34, 13, 34, 34, 33, 20, 20, 14, 12, 14, 17, 17, 14, 17, 19, 17, 14, 32, 14, 19, 17, 14, 14, 19, 17, 3, 38, 3, 17, 17, 12, 17, 17, 19, 14, 8, 14, 1, 10, 1, 10, 1, 10, 1, 10, 1, 10, 1, 1, 1, 1, 16, 1, 17, 19, 14, 2, 29, 10, 33, 21, 24, 24, 3, 24, 10, 24, 21, 27, 24, 24, 24, 10, 4, 33, 14, 4, 14, 33, 14, 24, 4, 29, 14, 27, 14, 33, 27, 10, 14, 24, 24, 3, 33, 10, 11, 8, 4, 10, 11, 3, 11, 3, 11, 3, 11, 3, 11, 3, 11, 20, 29, 29, 14, 3, 24, 29, 3, 29, 24, 29, 10, 24, 24, 3, 10, 29, 

In [106]:
timeAll=[sceneTimeVis,sceneTime,sceneTime1,sceneTime2,sceneTime_Time,sceneTime1_Time,sceneTime2_Time]
ep=[]
indx=[]
boundryindx=[]
for j in timeAll:
    for i in range(len(j)):
        if abs(min(manScene_boundry, key=lambda x:abs(x-j[i]))-j[i])<=10:
            ep.append(1)
            indx.append(np.argmin(np.abs(np.array(manScene_boundry)-j[i])))
            boundryindx.append(i)
        else:
            ep.append(0)
    #for i in range(len(indx)):
        #print(manScene_boundry[indx[i]],j[boundryindx[i]])
    truValues=ep.count(1)
    falseValues=ep.count(0)

    print(round(truValues/len(ep),2))

0.4
0.31
0.31
0.3
0.31
0.29
0.27


In [ ]:
ep=[]
indx=[]
boundryindx=[]
for i in range(len(sceneTimeVis)):
    if abs(min(manScene_boundry, key=lambda x:abs(x-sceneTimeVis[i]))-sceneTimeVis[i])<=10:
        ep.append(1)
        indx.append(np.argmin(np.abs(np.array(manScene_boundry)-sceneTimeVis[i])))
        boundryindx.append(i)
    else:
        ep.append(0)
for i in range(len(indx)):
    print(manScene_boundry[indx[i]],sceneTimeVis[boundryindx[i]])

truValues=ep.count(1)
falseValues=ep.count(0)

truValues/len(ep)

In [69]:
len(sceneTimeVis),len(sceneTime),len(sceneTime_Time)

(0, 92, 38)

In [30]:
ep=[]
indx=[]
boundryindx=[]
for i in range(len(sceneTime_Time)):
    if abs(min(manScene_boundry, key=lambda x:abs(x-sceneTime_Time[i]))-sceneTime_Time[i])<=10:
        ep.append(1)
        indx.append(np.argmin(np.abs(np.array(manScene_boundry)-sceneTime_Time[i])))
        boundryindx.append(i)
    else:
        ep.append(0)
for i in range(len(indx)):
    print(manScene_boundry[indx[i]],sceneTime_Time[boundryindx[i]],i)

truValues=ep.count(1)
falseValues=ep.count(0)

truValues/len(ep)

212.527 220.96 0
522.49 513.2 1
907.972 901.8000000000001 2
1172.0 1165.72 3
1172.0 1181.8 4
1275.53 1272.8 5
1474.84 1466.3600000000001 6
1474.84 1476.96 7
1520.84 1514.2 8
1912.61 1907.8400000000001 9
1912.61 1920.8 10
2642.48 2639.7200000000003 11
2805.0 2807.68 12
2856.44 2855.36 13
2894.6 2895.08 14
3249.32 3244.08 15
3315.65 3318.08 16


0.16666666666666666

(31, 108, 31)

In [17]:
manAnnotation='/people/berhe/Bureau/TLP_thesis/Scenes/'
annFileName='groundtruth'+'0'+str(1)+'.seg'
Df=pd.read_csv(manAnnotation+annFileName)

In [18]:
Df

,Unnamed: 0,start_time,end_time,story_id
0,0,0.0,6445.0,NaN
1,1,6445.0,106809.0,NaN
2,2,106809.0,212527.0,1.0
3,3,212527.0,292127.0,1.0
4,4,292127.0,389609.0,0.0
5,5,292127.0,389609.0,3.0
6,6,389609.0,522490.0,0.0
7,7,389609.0,522490.0,3.0
8,8,522490.0,668490.0,0.0
9,9,668490.0,770727.0,2.0


In [3]:
manBound=trSegem.load_ManAnnotation2()

In [4]:
print(manBound)

0        6445.0
1      106809.0
2      212527.0
3      292127.0
4      389609.0
5      389609.0
6      522490.0
7      522490.0
8      668490.0
9      770727.0
10     770727.0
11     907972.0
12     907972.0
13    1126880.0
14    1126880.0
15    1172000.0
16    1275530.0
17    1275530.0
18    1474840.0
19    1520840.0
20    1720960.0
21    1912610.0
22    2025280.0
23    2057560.0
24    2135960.0
25    2245640.0
26    2297920.0
27    2401160.0
28    2530800.0
29    2642480.0
30    2805000.0
31    2856440.0
32    2894600.0
33    3143120.0
34    3174160.0
35    3249320.0
36    3315650.0
Name: end_time, dtype: float64


In [76]:
print(dist_AvgFeat)

[[1.         0.34158266 0.15408587 ... 0.50457484 0.5155937  0.47146666]
 [0.34158266 1.         0.21950734 ... 0.26449972 0.39501715 0.37232488]
 [0.15408587 0.21950734 1.         ... 0.12519377 0.15069646 0.14693147]
 ...
 [0.50457484 0.26449972 0.12519377 ... 1.         0.52146065 0.39267492]
 [0.5155937  0.39501715 0.15069646 ... 0.52146065 1.         0.9216991 ]
 [0.47146666 0.37232488 0.14693147 ... 0.39267492 0.9216991  1.        ]]


In [79]:
print(dist_AvgFeat/2)

[[0.5        0.17079133 0.07704294 ... 0.25228742 0.25779685 0.23573333]
 [0.17079133 0.5        0.10975367 ... 0.13224986 0.19750857 0.18616244]
 [0.07704294 0.10975367 0.5        ... 0.06259689 0.07534823 0.07346573]
 ...
 [0.25228742 0.13224986 0.06259689 ... 0.5        0.26073033 0.19633746]
 [0.25779685 0.19750857 0.07534823 ... 0.26073033 0.5        0.46084955]
 [0.23573333 0.18616244 0.07346573 ... 0.19633746 0.46084955 0.5       ]]
